In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib

# Load model yang sudah dilatih sebelumnya (SVC1, SVC2, SVC3)
svc1 = joblib.load("svc1_node1_model.pkl")  
svc2 = joblib.load("svc2_Node2_model.pkl")  
svc3 = joblib.load("svc3_Node3_model.pkl")  

# Membuat meta-model (Generalizer) dengan Logistic Regression
meta_model = LogisticRegression()

# Membuat ensemble stacking classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('svc1', svc1),
        ('svc2', svc2),
        ('svc3', svc3)
    ],
    final_estimator=meta_model
)

# Load dataset
data = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
data.copy()
data_filled = data.fillna(0)

# Ekstraksi fitur dan label
X = data_filled[[' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', 'Flow Bytes/s', ' Flow Packets/s']].values
y = data_filled[' Label']

# Ganti nilai infinity dengan 0
X = np.where(np.isinf(X), 0, X)

# Split data untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Imputasi nilai kosong
imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Standardisasi data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Generate meta-features (prediksi dari svc1, svc2, svc3)
try:

    svc1_train_predictions = svc1.predict(X_train)
    svc2_train_predictions = svc2.predict(X_train)
    svc3_train_predictions = svc3.predict(X_train)

    print("SVC1 predictions (train):", svc1_train_predictions[:10])  
    print("SVC2 predictions (train):", svc2_train_predictions[:10])  
    print("SVC3 predictions (train):", svc3_train_predictions[:10])  

    # Prediksi dari nodes (Data Test)
    meta_features_train = np.column_stack([
        svc1_train_predictions,  
        svc2_train_predictions,  
        svc3_train_predictions,  
    ])
    print("Meta features (train):", meta_features_train[:10])  

    svc1_test_predictions = svc1.predict(X_test)
    svc2_test_predictions = svc2.predict(X_test)
    svc3_test_predictions = svc3.predict(X_test)

    print("SVC1 predictions (test):", svc1_test_predictions[:10])
    print("SVC2 predictions (test):", svc2_test_predictions[:10])
    print("SVC3 predictions (test):", svc3_test_predictions[:10])

    # Prediksi dari nodes (Data Uji)
    meta_features_test = np.column_stack([
        svc1_test_predictions,   
        svc2_test_predictions,   
        svc3_test_predictions,  
    ])
    print("Meta features (test):", meta_features_test[:10])  

    # Training meta-model (Logistic Regression)
    meta_model.fit(meta_features_train, y_train)

    # Evaluasi meta-model
    y_pred_meta = meta_model.predict(meta_features_test)

    print("Classification Report (Meta-Model):")
    print(classification_report(y_test, y_pred_meta))
    print("F1 Score (Meta-Model):", f1_score(y_test, y_pred_meta, average='weighted'))
    print("Accuracy Score (Meta-Model):", accuracy_score(y_test, y_pred_meta))

    # Save meta-model (Generalizer)
    joblib.dump(meta_model, "meta_model.pkl")
    print("Meta-model has been saved as 'meta_model.pkl'")

except Exception as e:
    print("Error during meta-model training or evaluation:", str(e))



SVC1 predictions (train): [0 0 1 0 1 0 1 1 1 1]
SVC2 predictions (train): [0 0 1 0 1 0 1 1 1 1]
SVC3 predictions (train): [0 0 1 0 1 1 1 1 1 1]
Meta features (train): [[0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
SVC1 predictions (test): [1 0 0 1 0 1 1 1 0 0]
SVC2 predictions (test): [1 0 0 1 0 1 1 1 0 0]
SVC3 predictions (test): [1 0 0 1 0 1 1 1 0 0]
Meta features (test): [[1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]]
Classification Report (Meta-Model):
              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00     39152
        DDoS       1.00      1.00      1.00     51146

    accuracy                           1.00     90298
   macro avg       1.00      1.00      1.00     90298
weighted avg       1.00      1.00      1.00     90298

F1 Score (Meta-Model): 0.9975192430261999
Accuracy Score (Meta-Model): 0.9975193249019911
Meta-model has been saved as 'meta_